In [1]:
require 'bio'

true

In [3]:
# Retrieve sequences.
fasta_file = Bio::FlatFile.auto("./data/TAIR10_cds_20101214_updated.fa")

#<Bio::FlatFile:0x000055cc83f3f4d0 @stream=#<Bio::FlatFile::BufferedInputStream:0x000055cc83f3f520 @io=#<File:./data/TAIR10_cds_20101214_updated.fa>, @path="./data/TAIR10_cds_20101214_updated.fa", @buffer=">AT1G51370.2 | Symbols:  | F-box/RNI-like/FBD-like domains-containing protein | chr1:19045615-19046748 FORWARD LENGTH=1041\nATGGTGGGTGGCAAGAAGAAAACCAAGATATGTGACAAAGTGTCACATGAGGAAGATAGGATAAGCCAGTTACCGGAACC\n">, @dbclass=Bio::FastaFormat, @splitter=#<Bio::FlatFile::Splitter::Default:0x000055cc83f3e008 @dbclass=Bio::FastaFormat, @stream=#<Bio::FlatFile::BufferedInputStream:0x000055cc83f3f520 @io=#<File:./data/TAIR10_cds_20101214_updated.fa>, @path="./data/TAIR10_cds_20101214_updated.fa", @buffer=">AT1G51370.2 | Symbols:  | F-box/RNI-like/FBD-like domains-containing protein | chr1:19045615-19046748 FORWARD LENGTH=1041\nATGGTGGGTGGCAAGAAGAAAACCAAGATATGTGACAAAGTGTCACATGAGGAAGATAGGATAAGCCAGTTACCGGAACC\n">, @entry_pos_flag=nil, @delimiter="\n>", @header=nil, @delimiter_overrun=1>, @skip_lead